In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab

Mounted at /content/drive
/content/drive/MyDrive/Colab


In [ ]:
!pip install datasets
!pip install transformers
!pip install tensorflow
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.0
    Uninstalling huggingface-hub-0.19.0:
      Successfully uninstalled huggingface-hub-0.19.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 8.0 MB/s eta 0:00:00


In [ ]:
!pip install wandb
import wandb
wandb.login()
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'val_f1_score',
    'goal': 'maximize'
    }

sweep_config['metric'] = metric

parameters_dict = {}

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0.00001 and 0.0001
        'distribution': 'uniform',
        'min': 0.000001,
        'max': 0.000006,
    },
    'decay_rate': {
        # a flat distribution between 0.5 and 0.1
        'distribution': 'uniform',
        'min': 0.05,
        'max': 0.1,
    },
    'batch_size': {
        # integers between 24 and 32
        # with evenly-distributed logarithms
        'distribution': 'q_log_uniform_values',
        'q': 2,
        'min': 24,
        'max': 32,
    },
    'dropout_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1,
    },
    'hidden_size': {
        # integers between 768 and 1024
        # with evenly-distributed logarithms
        'distribution': 'q_log_uniform_values',
        'q': 128,
        'min': 512,
        'max': 768,
    }
})

parameters_dict.update({
    'epochs': {
        'value': 3},
    'activation': {
        'value': 'relu'}
    })

sweep_config['parameters'] = parameters_dict

sweep_id = wandb.sweep(sweep_config, project="multilabel-text-emotion-classification")


Create sweep with ID: te49n4qc
Sweep URL: https://wandb.ai/huyyyle/multilabel-text-emotion-classification/sweeps/te49n4qc


In [ ]:
import argparse
import datasets
import pandas
import transformers
import tensorflow as tf
import numpy as np
import tensorflow_addons as tfa
from transformers import TFDistilBertModel

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-uncased")
def tokenize(examples):
    """Converts the text of each example to "input_ids", a sequence of integers
    representing 1-hot vectors for each token in the text"""
    return tokenizer(examples["text"], truncation=True, max_length=64,
                     padding="max_length", return_tensors="tf")

In [ ]:
def create_model(config):
    # Load the DistilBERT model
    distilbert = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

    # Define the input layer
    input_ids = tf.keras.layers.Input(shape=(64,), dtype=tf.int32, name='input_ids')
    outputs = distilbert.distilbert(input_ids)
    last_hidden_state = outputs.last_hidden_state
    cls_token = last_hidden_state[:, 0, :]

    # Add a dense layer with 'tanh' activation
    pooled_output = tf.keras.layers.Dense(units=config.hidden_size, activation=config.activation)(cls_token)

    # Add a dropout layer
    dropout = tf.keras.layers.Dropout(config.dropout_rate)(pooled_output)

    # Add the output layer for emotion classification
    emotion_classifier = tf.keras.layers.Dense(7, activation='sigmoid')(dropout)

    # Define the model
    model = tf.keras.Model(inputs=input_ids, outputs=emotion_classifier)

    # Compile the model
    optimizer = tf.keras.optimizers.AdamW(learning_rate=config.learning_rate, weight_decay=config.decay_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[tfa.metrics.F1Score(num_classes=7, average='micro', threshold=0.5)])

    #model.summary()

    return model

In [ ]:
def train(config, model_path="model", train_path="train.csv", dev_path="dev.csv"):
    # load the CSVs into Huggingface datasets to allow use of the tokenizer
    hf_dataset = datasets.load_dataset("csv", data_files={
        "train": train_path, "validation": dev_path})

    # the labels are the names of all columns except the first
    labels = hf_dataset["train"].column_names[1:]

    def gather_labels(example):
        """Converts the label columns into a list of 0s and 1s"""
        # the float here is because F1Score requires floats
        return {"labels": [float(example[l]) for l in labels]}

    # convert text and labels to format expected by model
    hf_dataset = hf_dataset.map(gather_labels)
    hf_dataset = hf_dataset.map(tokenize, batched=True)

    # TODO: Define the model
    model = create_model(config)  # Ensure create_model() is updated for DistilBERT

    # TODO: Prepare the data for training
    train_dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': hf_dataset['train']['input_ids'],
        },
        hf_dataset['train']['labels']
    )).shuffle(100).batch(config.batch_size)

    validation_dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': hf_dataset['validation']['input_ids'],
        },
        hf_dataset['validation']['labels']
    )).batch(config.batch_size)

    # TODO: Train the model
    model.fit(train_dataset, validation_data=validation_dataset, epochs=config.epochs,
              callbacks=[
            tf.keras.callbacks.ModelCheckpoint(
                filepath=model_path,
                monitor="val_f1_score",
                mode="max",
                save_best_only=True),
            WandbMetricsLogger(log_freq=5),
            WandbModelCheckpoint("models")
            ])


In [11]:
def train_wandb(config=None):
  with wandb.init(config=config):
    # If called by wandb.agent, as below,
    # this config will be set by Sweep Controller
    config = wandb.config
    train(config)


wandb.agent(sweep_id, train_wandb, count=3)


wandb: Agent Starting Run: hdk7iltk with config:
wandb: 	batch_size: 96
wandb: 	decay_rate: 0.08643623630358307
wandb: 	dropout_rate: 0.08915880971111093
wandb: 	epochs: 3
wandb: 	hidden_size: 512
wandb: 	learning_rate: 2.2739927361986424e-06


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Epoch 1/3
263/263 [==============================] - ETA: 0s - loss: 0.2458 - f1_score: 0.0036

wandb: Adding directory to artifact (./models)... Done. 2.5s


263/263 [==============================] - 217s 753ms/step - loss: 0.2458 - f1_score: 0.0036 - val_loss: 0.2053 - val_f1_score: 0.0000e+00
Epoch 2/3
263/263 [==============================] - ETA: 0s - loss: 0.1696 - f1_score: 0.3475

...

wandb: Adding directory to artifact (./models)... Done. 4.1s


630/630 [==============================] - 206s 328ms/step - loss: 0.1141 - f1_score: 0.6560 - val_loss: 0.0779 - val_f1_score: 0.8063
Epoch 3/3
630/630 [==============================] - ETA: 0s - loss: 0.0791 - f1_score: 0.7997

wandb: Adding directory to artifact (./models)... Done. 4.4s


630/630 [==============================] - 209s 332ms/step - loss: 0.0791 - f1_score: 0.7997 - val_loss: 0.0683 - val_f1_score: 0.8327


batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
batch/f1_score,▂▂▁▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▅▆▆▆▆▆▆▇▇█████████████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▆▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,▁▅█
epoch/f1_score,▁▇█
epoch/learning_rate,▁▁▁
epoch/loss,█▃▁
epoch/val_f1_score,▁██
epoch/val_loss,█▂▁
batch/batch_step,1885


In [ ]:
from google.colab import runtime
runtime.unassign()